Linear Data Lab 11

Original lab written by: Emily J. King

Goals: Recognize connection between eigenvalues, determinant, rank, and trace. Correctly interpret output of computer-calculated eigenvalues/eigenvectors. Understand power iteration. Practice applications of spectral graph theory.

Additional files needed: poweriter.py, Linear_Data_Chapter_2_Lab.pdf

In [ ]:
import numpy as np
from numpy.linalg import norm
from numpy.linalg import det
from numpy.linalg import matrix_rank as rank 
from numpy.linalg import eigvals
from numpy.linalg import eig
from numpy.linalg import inv
from poweriter import poweriter

Section 1: Basics of eigen theory

Eigenvalues of diagonal matrices

Recall what multiplying by a diagonal matrix does: it scales the first coordinate by the first diagonal element, the second coordinate by the second diagonal element, and so on.  So, what should the eigenvalues and eigenvectors be?  Discuss.

Now we test by making a 5x5 diagonal matrix with small integer entries.

In [ ]:
F=np.diag(np.array([1,-1,2,-2,3]))
F

What should the trace, determinant, and rank be?

For any matrix, the trace is just the sum of the diagonal elements.  For a diagonal matrix, the determinant is the product of the diagonal elements.  For a diagonal matrix, the rank is the number of non-zero diagonal elements.  Test this.

In [ ]:
np.trace(F)

In [ ]:
det(F)

In [ ]:
rank(F)

Recall what multiplying by a diagonal matrix does: it scales the first coordinate by the first diagonal element, the second coordinate by the second diagonal element, and so on.  So, what should the eigenvalues and eigenvectors be?  Discuss.

Let's compute the eigenvalues alone.

In [ ]:
eigvals(F)

Notice that the eigenvalues are simply the diagonal elements of the diagonal matrix, but they may be listed in a different order due to the algorithm used to compute them.  

Now let's see the command to compute the eigenvalues and eigenvectors.

In [ ]:
d, U = eig(F)
d

In [ ]:
U

Notice the eig command returns an array and a matrix. The array is the same thing returned by eigvals. When possible, the matrix returned is an orthogonal matrix: that is, the columns form an orthonormal basis for R^n. Otherwise, the columns are unit norm but not necessarily orthogonal. The columns of the matrix are eigenvectors corresponding to the eigenvalues listed in the array.  IF it is possible to decompose a matrix in the form UDU^(-1) that we've played around with for the last 3 labs, then U above and D formed as the diagonal matrix with diagonal d above, will work.

Let's test.

In [ ]:
np.allclose(U@np.diag(d)@inv(U),F)

What happens when we multiply F times a column of U?  Let's test the first column.

In [ ]:
F@U[:,0]

Now let's scalar multiply the first column by the first element of d.  They are the same.

In [ ]:
d[0]*U[:,0]

Let's test that F times the 3rd column of U is the scalar multiple of the third element of d times the 3rd column of U.

In [ ]:
j=2
np.allclose(F@U[:,j],d[j]*U[:,j])

Let A be a random 5x5 matrix and define M=AFA^(-1).

In [ ]:
A=np.random.rand(5,5)
M=A@F@inv(A)

What are the trace, determinant, and rank of the new matrix M?

In [ ]:
np.trace(M)

In [ ]:
det(M)

In [ ]:
rank(M)

They are all (up to floating point arithmetic) the same as for F. For any matrix B and invertible matrix C, B and CBC^(-1) always have the same trace, determinant, and rank.  And if B is diagonal, then it is easy to compute those by hand.

Now let's compute the eigenvalues and some eigenvectors.

In [ ]:
d_M, U_M = eig(M)
d_M

The eigenvalues are the same (possibly in a different order and up to floating point arithmetic) as F.  This is because for ny matrix B and invertible matrix C, B and CBC^(-1) always have the eigenvalues (but not in general the same eigenvectors).  What are the eigenvectors?

In [ ]:
U_M

What is the relationship between U_M and A?  The columns of both matrices are eigenvectors for M.  Let's compare the diagonal of F and the rounded eigenvalues returned by the command.

In [ ]:
np.diag(F)

In [ ]:
np.round(np.real((d_M)))

For any eigenvalue that appears just once, the column of U_M that corresponds to that entry of d_M is plus or minus 1 times the normalization of the column of A that corresponds to that entry of the diagonal of F.  Test this, fixing indices based on your actual matrices.

If an eigenvalue is repeated, then the corresponding columns of U_M are an orthonormal basis for the span of the columns of A matching that eigenvalue. 

In [ ]:
fj=1
dj=1
np.allclose(A[:,fj]/norm(A[:,fj]),U_M[:,dj])

In [ ]:
np.allclose(-A[:,fj]/norm(A[:,fj]),U_M[:,dj])

Exactly one of the two commands above should be true.

Eigenvectors of non-diagonalizable matrices

Let's now consider a matrix that cannot be written in in the form CDC^(-1) for C invertible and D diagonal, a shear matrix.

Let S be a 2x2 matrix that shears by a factor of 3 downwards.  What do we know must stay in place?

In [ ]:
S=np.array([[1, 0],[-3, 1]])
S

Now, we are going to compute the eigenvalues and eigenvectors of S.


In [ ]:
d, U = eig(S)
d

Note that there are 2 ones. Does that mean that there are two dimensions of vectors in R^2 fixed by shearing?  Does that make sense?


In [ ]:
U

Now, we can hopefully see what has happened.  Both columns of U are the same (up to floating point arithmetic).  In particular, that means that U is not invertible, and thus UDU^(-1) isn't defined.  The issue is that S is 2x2 but only has one dimension of eigenvectors: the y-axis.  Said another way: S only has one eigenspace, for eigenvalue 1, and that eigenspace is only 1-dimensional.  But numpy will always return d eigenvalues and d eigenvectors for a dxd matrix.

Section 2: Power iteration

When a matrix is "nice enough", if you take a random vector and multiply it by higher and higher powers of the matrix, you get output that points closer and closer to the direction of an eigenvector for the largers eigenvalue.  We saw this a bit when playing around with the transition matrix: higher powers applied to the two starting vectors we tried (sunny today vs. gray today) always emphasized the steady state vector.  

We can do this for other matrices which have largest eigenvalue larger than 1. To make sure that the vector norms aren't getting too big for the computer, we can normalize.  So, the idea is, multiply a random vector by our matrix, normalize this output, then mulitply that by the matrix and repeat.  Take a look at the code in poweriter.py and discuss.

Let's try this with the transition matrix from lecture:

In [ ]:
P=np.array([[9/10, 1/2],[1/10, 1/2]])
xout=poweriter(P,np.random.rand(2),1000)
xout

Now let's compare the normalization of the vector we know to be the steady state vector (i.e., a vector with eigenvalue 1, while the other eigenvalue is 2/5 < 1) to the output:

In [ ]:
np.allclose(xout,np.array([5/6, 1/6])/norm(np.array([5/6, 1/6])))

In [ ]:
np.allclose(xout,-np.array([5/6, 1/6])/norm(np.array([5/6, 1/6])))

We can see that power iteration output the normalization of the steady state vector (or negative the normalization).

Now let's do the same thing with the matrix M above. 

In [ ]:
xout=poweriter(M,np.random.rand(5),1000)

This returns (an approximation of) an eigenvector of M with the largest eigenvalue (in absolute value). Let's test.

In [ ]:
M@xout/xout

Section 3: Playing with spectral graph theory

Computing the graph Laplacian of the (unweighted, undirected) graph from Linear_Data_Chapter_2_Lab.pdf and the book.  First the adjacency matrix.

In [ ]:
A = np.array([[0, 1, 0, 0, 1, 0], [1, 0, 1, 0, 1, 0], [0, 1, 0, 1, 0, 0], [0, 0, 1, 0, 1, 1], [1, 1, 0, 1, 0, 0], [0, 0, 0, 1, 0, 0]])
A

Then the degree matrix.

In [ ]:
D=np.diag(A@np.ones(A.shape[0]))
D

And finally, the (unnormalized) graph Laplacian.

In [ ]:
L=D-A
L

Let's compute the eigenvalues and eigenvectors.

In [ ]:
d_L, U_L = eig(L)
d_L

Notice that there is (up to floating point arithmetic) exactly one zero as an eigenvalue since the graph has exactly one connected component.  Also, since the graph Laplacian is a special kind of matrix called positive semidefininte, the eigenvalues are all non-negative real numbers. 

Let's look at the entries of the Fielder eigenvector (i.e. the returned eigenvector corresponding to the smallest positive eigenvalue).  If the second element of d_L isn't the smallest positive eigenvalue, adjust the code below.

In [ ]:
U_L[:,1]

This suggests clustering vertices 1, 2, 3, and 5 together and 4 and 6 together.

Now, let's looks at the ratio between the largest eigenvalue and the Fiedler eigenvalue (i.e., the smallest positive eigenvalue).  We'll track changes to this number as we manipulate the graph.

In [ ]:
d_L[1]/d_L[5]

Let's add an edge that doesn't yet exist in {1,2,3,5}, 1<->3. And then compute the eigenvalues and eigenvectors as well as the ratio between the largest eigenvalue and the Fielder eigenvalue.

In [ ]:
A[0,2]=1
A[2,0]=1
L=np.diag(A@np.ones(A.shape[0]))-A
L

In [ ]:
d_L, U_L = eig(L)
d_L

Let's look at the entries of the Fielder eigenvector, double checking where the smallest non-zero eigenvalue is.

In [ ]:
U_L[:,1]

This still suggests clustering vertices 1, 2, 3, and 5 together and 4 and 6 together.

Now, let's looks at the ratio between the largest eigenvalue and the Fiedler eigenvalue.  For me, the largest is at the 3rd position.

In [ ]:
d_L[1]/d_L[2]

Notice that this ratio is slightly smaller, suggesting stronger connectivity within our predicted clusters.

Let's add an edge 1<->6, which is "across" the previously predicted clusters.

In [ ]:
A[0,5]=1
A[5,0]=1
L=np.diag(A@np.ones(A.shape[0]))-A
L

In [ ]:
d_L, U_L = eig(L)
d_L

Let's look at the entries of the Fielder eigenvector. For me, the smallest positive eigenvalue is in the fifth position.  Change as necessary.

In [ ]:
U_L[:,4]

This still suggests clustering vertices 1, 2, 3, and 5 together and 4 and 6 together.

But let's looks at the ratio between the largest eigenvalue and the Fiedler eigenvalue.  For me, the largest is at the second position.

In [ ]:
d_L[4]/d_L[1]

The ratio is larger, suggesting that the clustering is less certain.

Play around with other graphs to see how connectivity affects the spectrum of the graph Laplacian.

Exercises

1a. Make a random symmetric 3x3 matrix using the code below.  (A symmetrix matrix is equal to its own transpose.)

In [ ]:
C=np.random.rand(3,3)
C=C+np.transpose(C)
C

b. Run power iteration to find the largest eigenvalue and a corresponding eigenvector.

c. Use numpy commands to directly find the eigenvectors and eigenvalues.


2a. Enter the adjacency matrix of the weighted graph from Linear_Data_Chapter_2_Lab.pdf.

b. Compute the (unnormalized) graph Laplacian.

c. Cluster the graph using the Fielder eigenvector.

(Interpret the Fielder eigenvector here)